# Database Engineering

In [26]:
from os import path, remove
from datetime import datetime
import pandas as pd
from sqlalchemy import (
    create_engine, 
    Column, ForeignKey,
    Integer, String, Date, Float)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

_MEASUREMENT_CSV = path.join('Resources', 'cleaned_hawaii_measurements.csv')
_STATION_CSV = path.join('Resources', 'hawaii_stations.csv')
_SQLITE_FILE = 'hawaii.sqlite'

Base = declarative_base()

class Station(Base):
    __tablename__ = 'station'
    station = Column(String, primary_key=True)
    name = Column(String)
    latitude = Column(Float(asdecimal=True))
    longitude = Column(Float(asdecimal=True))
    elevation = Column(Float(asdecimal=True))

class Measurement(Base):
    __tablename__ = 'measurement'
    station = Column(String,
                     #ForeignKey('station.station'), # Got issue in Base.prepare(engine, reflect=True). Might come back to this
                     primary_key=True)
    date = Column(Date, primary_key=True)
    prcp = Column(Float(asdecimal=True))
    tobs = Column(Integer)

# Handle removing file independent of OS
if path.isfile(_SQLITE_FILE): 
    remove(_SQLITE_FILE)
engine = create_engine(f'sqlite:///{_SQLITE_FILE}')
Base.metadata.create_all(engine)

# Read data into DataFrames then write into sqlite
session = Session(engine)
session.add_all([
    Station(**s)
    for s in pd.read_csv(_STATION_CSV).to_dict(orient='records')
] + [
    Measurement(**m)
    for m in pd.read_csv(_MEASUREMENT_CSV).assign(
        date=lambda x: pd.to_datetime(x['date']) # Sqlite requires date type
    ).to_dict(orient='records')
])
session.commit()
session.close()